created on 11/26/15 (thanksgiving; bay is in taiwan; yesterday was my last day at clarapath; working on this to refresh scala)

In [1]:
import scala.io.Source

import scala.io.Source

In [2]:
// source.fromfile creates a generator

val text_0 = Source.fromFile("unlabeledTrainData.tsv")

text_0: scala.io.BufferedSource = non-empty iterator

In [3]:
// read into memory by converting generator into a string

val text_1 = text_0.mkString

text_1: String = """
id	review
"9999_0"	"Watching Time Chasers, it obvious that it was made by a bunch of friends. Maybe they were sitting around one day in film school and said, \"Hey, let's pool our money together and make a really bad movie!\" Or something like that. What ever they said, they still ended up making a really bad movie--dull story, bad script, lame acting, poor cinematography, bottom of the barrel stock music, etc. All corners were cut, except the one that would have prevented this film's release. Life's like that."
"45057_0"	"I saw this film about 20 years ago and remember it as being particularly nasty. I believe it is based on a true incident: a young man breaks into a nurses' home and rapes, tortures and kills various women.<br /><br />It is in black and white but saves the colour for one shocking shot.<br /><br />At the end the film seems to be trying to make some political statement but it just comes across as confused and obscene.<br /><br />Avoid."
"15561_0"	"M

In [4]:
// split by tabs

val text_2 = text_1.split("\t")

text_2: scala.Array[java.lang.String] = Array(
  "id",
  """
  review
  "9999_0"
  """,
  """
  "Watching Time Chasers, it obvious that it was made by a bunch of friends. Maybe they were sitting around one day in film school and said, \"Hey, let's pool our money together and make a really bad movie!\" Or something like that. What ever they said, they still ended up making a really bad movie--dull story, bad script, lame acting, poor cinematography, bottom of the barrel stock music, etc. All corners were cut, except the one that would have prevented this film's release. Life's like that."
  "45057_0"
  """,
  """
...

In [5]:
// there should be 50k lines in this file

text_2.length

res4: Int = 50002

In [6]:
// convert each line to lowercase

val text_3 = text_2.map(line => line.toLowerCase)

text_3: scala.Array[java.lang.String] = Array(
  "id",
  """
  review
  "9999_0"
  """,
  """
  "watching time chasers, it obvious that it was made by a bunch of friends. maybe they were sitting around one day in film school and said, \"hey, let's pool our money together and make a really bad movie!\" or something like that. what ever they said, they still ended up making a really bad movie--dull story, bad script, lame acting, poor cinematography, bottom of the barrel stock music, etc. all corners were cut, except the one that would have prevented this film's release. life's like that."
  "45057_0"
  """,
  """
...

In [7]:
// for each line, discard punctuation
// https://stackoverflow.com/questions/30074109/removing-punctuation-marks-form-text-in-scala-spark

val text_4 = text_3.map(line => line.replaceAll("""[\p{Punct}&&[^.]]""", ""))

text_4: scala.Array[java.lang.String] = Array(
  "id",
  """
  review
  99990
  """,
  """
  watching time chasers it obvious that it was made by a bunch of friends. maybe they were sitting around one day in film school and said hey lets pool our money together and make a really bad movie or something like that. what ever they said they still ended up making a really bad moviedull story bad script lame acting poor cinematography bottom of the barrel stock music etc. all corners were cut except the one that would have prevented this films release. lifes like that.
  450570
  """,
  """
...

In [8]:
text_4(34)

res7: java.lang.String = """
for years friends expressed amazement that i had never watched niptuck  just my kind of show they insisted. and so spurred by the recent assault of preseason publicity i decided to pay a firsttime visit to niptuck last night. i was expecting sly and witty satire. what i found instead in repeat episodes that focused on a serial mangler and a deranged racist was a puerile unrelenting torrent of violence and sadism. robotic andor hysterical acting moronic plot contortions and enough deep shadow in miami yet to make edgar ulmer spin like a top. it was like watching joel schumacher direct ricky martin as hannibal lecter. nice to know what my friends think of my taste.
361850
"""

## wordcount!

In [16]:
// slicing so we dont consume too much heap memory

val text_5 = text_4.slice(0, 100)

text_5: scala.Array[java.lang.String] = Array(
  "id",
  """
  review
  99990
  """,
  """
  watching time chasers it obvious that it was made by a bunch of friends. maybe they were sitting around one day in film school and said hey lets pool our money together and make a really bad movie or something like that. what ever they said they still ended up making a really bad moviedull story bad script lame acting poor cinematography bottom of the barrel stock music etc. all corners were cut except the one that would have prevented this films release. lifes like that.
  450570
  """,
  """
...

In [12]:
// this is an array of a single string, so just split by words
// now we have a single array of tokens

val text_6 = text_5.flatMap(line => line.split("\\s"))

text_6: scala.Array[java.lang.String] = Array(
  "id",
  "review",
  "99990",
  "watching",
  "time",
  "chasers",
  "it",
  "obvious",
  "that",
  "it",
  "was",
  "made",
  "by",
  "a",
...

In [13]:
// groupby tokens; now we have a map where k = token and v = array of said tokens

val text_7 =  text_6.groupBy(token => token)

text_7: scala.collection.immutable.Map[java.lang.String,scala.Array[java.lang.String]] = Map(
  "unlikely." -> Array("unlikely."),
  "incident" -> Array("incident", "incident", "incident", "incident"),
  "breaks" -> Array("breaks", "breaks", "breaks", "breaks"),
  "forgotten" -> Array("forgotten", "forgotten", "forgotten"),
  "tripe" -> Array("tripe"),
  "lover" -> Array("lover"),
  "of." -> Array("of."),
  "lead." -> Array("lead."),
  "terrible" -> Array("terrible", "terrible", "terrible"),
  "rate" -> Array("rate", "rate"),
...

In [14]:
// count length of array of tokens for word count

val text_8 = text_7.mapValues(v_in_kv => v_in_kv.size)

text_8: scala.collection.immutable.Map[java.lang.String,Int] = Map(
  "unlikely." -> 1,
  "incident" -> 4,
  "breaks" -> 4,
  "forgotten" -> 3,
  "tripe" -> 1,
  "lover" -> 1,
  "of." -> 1,
  "lead." -> 1,
  "terrible" -> 3,
  "rate" -> 2,
  "inevitable" -> 1,
  "nonetheless" -> 1,
  "lights" -> 1,
  "rage" -> 1,
...

In [15]:
// filter out kv pairs where v is small!
// https://stackoverflow.com/questions/11854433/filter-map-for-values-of-none

text_8.filter(kv => kv._2 > 4)

res14: scala.collection.immutable.Map[java.lang.String,Int] = Map(
  "looks" -> 5,
  "california" -> 5,
  "particularly" -> 6,
  "used" -> 8,
  "beautiful" -> 11,
  "funny" -> 7,
  "down" -> 6,
  "serial" -> 5,
  "please" -> 10,
  "interesting" -> 7,
  "" -> 77,
  "read" -> 6,
  "number" -> 5,
  "actor" -> 8,
...